In [36]:
import pandas as pd
import re

import torch
import torchtext
from torchtext import data,datasets
from torchtext.vocab import Vocab
import opendatasets as od

In [7]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
print(device)
print(torch.__version__)
print(torchtext.__version__)

cuda
2.1.0+cu118
0.16.0+cpu


In [15]:
od.download('https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset')

100%|██████████| 41.0M/41.0M [00:12<00:00, 3.33MB/s]


### Importing Dataset

In [21]:
real_df = pd.read_csv('./fake-and-real-news-dataset/True.csv')
fake_df = pd.read_csv('./fake-and-real-news-dataset/Fake.csv')

In [22]:
real_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [23]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


### Preprocessing and Merging

In [24]:
real_df.drop(['date','subject'],axis=1,inplace=True)
fake_df.drop(['date','subject'],axis=1,inplace=True)

In [26]:
real_df['Label'] = 1
fake_df['Label'] = 0

In [27]:
real_df['TEXT'] = real_df['title']+real_df['text']
real_df.drop(['title','text'],axis=1,inplace=True)

In [29]:
fake_df['TEXT'] = fake_df['title']+fake_df['text']
fake_df.drop(['title','text'],axis=1,inplace=True)

In [30]:
real_df.head(3)

,Label,TEXT
0,1,"As U.S. budget fight looms, Republicans flip t..."
1,1,U.S. military to accept transgender recruits o...
2,1,Senior U.S. Republican senator: 'Let Mr. Muell...


In [31]:
fake_df.head(3)

,Label,TEXT
0,0,Donald Trump Sends Out Embarrassing New Year’...
1,0,Drunk Bragging Trump Staffer Started Russian ...
2,0,Sheriff David Clarke Becomes An Internet Joke...


In [33]:
news_df = pd.concat([fake_df,real_df],ignore_index=True)
news_df

,Label,TEXT
0,0,Donald Trump Sends Out Embarrassing New Year’...
1,0,Drunk Bragging Trump Staffer Started Russian ...
2,0,Sheriff David Clarke Becomes An Internet Joke...
3,0,Trump Is So Obsessed He Even Has Obama’s Name...
4,0,Pope Francis Just Called Out Donald Trump Dur...
...,...,...
44893,1,'Fully committed' NATO backs new U.S. approach...
44894,1,LexisNexis withdrew two products from Chinese ...
44895,1,Minsk cultural hub becomes haven from authorit...
44896,1,Vatican upbeat on possibility of Pope Francis ...


In [34]:
news_df.rename(columns={'Label':'LABEL'},inplace=True)

### NLP Tasks

In [37]:
news_df['TEXT'] = news_df['TEXT'].apply(lambda x:x.lower())
news_df['TEXT'] = news_df['TEXT'].apply(lambda x:re.sub('[^a-zA-z0-9\s]','',x))

### RNN-LSTM Model Building

### Training and Evaluation